In [2]:
import pandas as pd
import numpy as np

In [3]:
#load data
location = 'C:/Users/andti/OneDrive/Documents/Datasets/WELFake_Dataset.csv'
d = pd.read_csv(location)
d.drop(['Unnamed: 0'], axis=1, inplace = True)

In [4]:
DataSet = d.copy()

Text wrangling and cleaning

In [5]:
#using text as an indicator 
TextDS = d[['text', 'label']]
#remove stop words
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def TextWrangle (DataElement):
    LowerCase = DataElement.str.lower()
    RemoveStopWords = LowerCase.astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return RemoveStopWords

TextDS['text'] = TextWrangle(TextDS['text'])

#remove nulls
NullLines = TextDS['text'].isnull()
TextDS = TextDS[~NullLines]

trainData = TextDS['text']
trainTarget = TextDS['label']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\andti\AppData\Local\Temp\ipykernel_12768\158800997.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TextDS['text'] = TextWrangle(TextDS['text'])


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

pipe = Pipeline([
    ('tfidf', TfidfVectorizer(
        sublinear_tf=True, 
        min_df=5, 
        norm='l2', 
        encoding='latin-1', 
        ngram_range=(1,2), 
        stop_words='english', 
        max_features=5000)),
        ('model', CalibratedClassifierCV(LinearSVC()))
])
pipe.fit(trainData, trainTarget)
